# LLM Zoomcamp 2024 Competition
This competition requires the use of LLMs to solve high school mathematics problems. The task is to develop models that can accurately solve the problems and then the predictions submitted.

In [1]:
# Import Open AI model.
from langchain_cohere import ChatCohere

/home/codespace/miniconda3/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# Import required modules.
import numpy as np
import pandas as pd
import os
from langchain_core.prompts import ChatPromptTemplate
import re
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from scorer import score

/home/codespace/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set up OpenAI API key.
load_dotenv()

True

In [4]:
question = "Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$."

In [5]:
# Define the application.
chat = ChatCohere(max_tokens=256, temperature=0)

In [6]:
messages = [
        SystemMessage(content="Please solve the following high school math problem and return the answer as float"),
        HumanMessage(content=question),
    ]
answer = chat.invoke(messages)
answer

AIMessage(content="The given expression is $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.\n\nFirst, let's calculate the ratio:\n\n$$\n\\begin{align*}\n    \\frac{17}{5} : \\frac{34}{3} & = \\frac{\\frac{17}{5}}{\\frac{34}{3}}\\\\\\\\\n    & = \\frac{17 \\cdot 3}{5 \\cdot 34}\\\\\\\\\n    & = \\frac{51}{170}\\\\\\\\\n    & = \\frac{3}{10}\n\\end{align*}\n$$\n\nNow, we can add the result to $1.3$:\n\n$$\n\\begin{align*}\n    \\frac{3}{10} + 1.3 & = \\frac{3}{10} + \\frac{13}{10}\\\\\\\\\n    & = \\frac{16}{10}\\\\\\\\\n    & = \\frac{8}{5}\\\\\\\\\n    & = \\boxed{1.6}\n\\end{align*}\n$$\n\nSo, the value of the given expression is $\\bf{1.6}$.", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '156cfdec-02ad-4b5b-bab9-0326481cd244', 'token_count': {'input_tokens': 253.0, 'output_tokens': 262.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': No

In [13]:
match = re.search(r'(So, the value of the given expression is $\\bf{(.+?)}$.",)', answer, re.IGNORECASE)
match

TypeError: expected string or bytes-like object

In [42]:
chat = ChatCohere()
messages = [
        SystemMessage(content="Please extract the number after 'So, the final answer is $\\boxed{' and '}$'"),
        HumanMessage(content=str(answer)),
    ]
answer2 = chat.invoke(messages)
answer2

AIMessage(content='1.6', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '6c308c47-529d-4bf9-bd12-18b0e7b792f8', 'token_count': {'input_tokens': 911.0, 'output_tokens': 3.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '6c308c47-529d-4bf9-bd12-18b0e7b792f8', 'token_count': {'input_tokens': 911.0, 'output_tokens': 3.0}}, id='run-399f1267-fbf8-4dca-96d8-84864fba52da-0', usage_metadata={'input_tokens': 911, 'output_tokens': 3, 'total_tokens': 914})

In [ ]:
# Load the train data.
df_train = pd.read_csv('data/train.csv')
df_train.head()

In [4]:
# Define the application.
llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=1500)

In [5]:
# Function to query the LLM.
def get_answer(question):
    messages = [
        SystemMessage(content="You are a mathematics genius, solve the expression and return answer in float or integer"),
        HumanMessage(content=question),
    ]

    parser = StrOutputParser()
    chain = llm | parser
    result = chain.invoke(messages)
    return result

In [6]:
# Function to return the answer.
def return_answer(text):
    # Regex pattern to match the last float or integer
    pattern = r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?\D*$'
    match = re.search(pattern, text)
    if match:
        # Extract the matched number
        last_number = match.group(0)
        # Remove any trailing non-numeric characters
        pattern2 =  r'\b\d+\.\d+|\b\d+'
        match2 = re.search(pattern2, last_number)
        if match2:
            cleaned_number = (match2.group())
            return float(cleaned_number)

In [7]:
# Function to prepare the dataset and get answers.
pool = ThreadPoolExecutor(max_workers=6)

def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [8]:
# Function for processing each row.
def process_row(row):
    problem_id = row['problem_id']
    problem_text = row['problem_text']

    llm_reasoning = get_answer(problem_text)

    numerical_answer = return_answer(llm_reasoning)

    return {
        'problem_id': problem_id,
        'problem_text': problem_text,
        'llm_reasoning': llm_reasoning,
        'answer': numerical_answer
    }

In [9]:
# Load the train data.
df_train = pd.read_csv('data/train.csv')
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [10]:
# Process row.
rows = df_train.head().to_dict(orient='records')
process_row(rows[0])

{'problem_id': 2374,
 'problem_text': 'Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.',
 'llm_reasoning': 'To solve the expression \\(\\dfrac{17}{5} : \\dfrac{34}{3} + 1.3\\), we need to follow the order of operations, which includes handling division and addition appropriately.\n\nFirst, let\'s rewrite the division part of the expression. The division symbol ":" can be interpreted as a fraction, so we have:\n\n\\[\n\\dfrac{17}{5} \\div \\dfrac{34}{3}\n\\]\n\nDividing by a fraction is equivalent to multiplying by its reciprocal. Therefore, we can rewrite the division as:\n\n\\[\n\\dfrac{17}{5} \\times \\dfrac{3}{34}\n\\]\n\nNext, we perform the multiplication of the fractions:\n\n\\[\n\\dfrac{17 \\times 3}{5 \\times 34} = \\dfrac{51}{170}\n\\]\n\nWe can simplify \\(\\dfrac{51}{170}\\) by finding the greatest common divisor (GCD) of 51 and 170. The GCD of 51 and 170 is 17. Therefore, we divide both the numerator and the denominator by 17:\n\n\\[\n\\dfrac{51 \\di

In [11]:
# Return results df_train head.
results = map_progress(pool, rows, process_row)
df_results = pd.DataFrame(results)
df_results

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.32s/it]


,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,To solve the expression \(\dfrac{17}{5} : \dfr...,1.6
1,4723,"In a company of 30 people, 25 use the social n...","To solve this problem, we need to use the prin...",24.0
2,7135,The number of road traffic accidents (RTAs) in...,To determine the percentage decrease in the nu...,32.0
3,5814,Find the value of the expression $\dfrac{2\str...,To solve the expression \(\dfrac{2^{-5} \cdot ...,256.0
4,9237,A traveler from Moscow wants to visit four cit...,"To solve this problem, we need to find a combi...",53.0


100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:20<00:00,  2.99s/it]

In [12]:
# Show df_train head.
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [13]:
# Score df_train head results.
score(df_train.head(), df_results, 'problem_id')

1.0

In [14]:
# Put all together in a function.
def prepare_prompts_and_get_answers(df):
    rows = df.to_dict(orient='records')
    results = map_progress(pool, rows, process_row)
    return pd.DataFrame(results)

In [15]:
# Return the results head.
df_train_results = prepare_prompts_and_get_answers(df_train)
df_train_results.head()

100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:36<00:00,  1.04it/s]


,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,To solve the expression \(\dfrac{17}{5} : \dfr...,1.6
1,4723,"In a company of 30 people, 25 use the social n...","To solve this problem, we need to use the prin...",24.0
2,7135,The number of road traffic accidents (RTAs) in...,To determine the percentage decrease in the nu...,32.0
3,5814,Find the value of the expression $\dfrac{2\str...,To find the value of the expression \(\dfrac{2...,256.0
4,9237,A traveler from Moscow wants to visit four cit...,"To solve this problem, we need to find a combi...",53.0


 95%|████████████████████████████████████████████████████████████████████████████████████▌    | 95/100 [01:08<00:05,  1.02s/it]

In [16]:
# Get the total score.
score(df_train, df_train_results, 'problem_id')

0.7

In [17]:
# Prepare the test submission.
df_test = pd.read_csv('data/test.csv')

df_test_results = prepare_prompts_and_get_answers(df_test)

submission = df_test_results[['problem_id', 'answer']]
submission.to_csv('starter_notebook_submission_gpt_4o.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:20<00:00,  1.24it/s]


In [18]:
train_result = df_train_results[['problem_id', 'answer']]
train_result.to_csv('train_result_gpt_4o.csv', index=False)